https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df

https://www.kaggle.com/deshwalmahesh/nlp-beginner-1-rnn-lstm-gru-embeddings-glove

In [1]:
from tqdm import tqdm, tqdm_notebook
import numpy as np
import os

from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from sklearn.feature_extraction.text import CountVectorizer
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm, tqdm_notebook

random_state = 4
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import json
import time

from utils import *
from models import *
from training import *

In [2]:
path = 'data/16S-SG-reads.fa/16S-reads.fa'
#Preprocessing
# Loading, Cleaning and Labelling 
sg, tax_dict = preprocessing(path, size = 8)
# Encoding
sg_encoded = encoder(sg)

# Dataset

#### Train and Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(np.array(list(sg_encoded['encoded'])), 
                                                    np.array(list(sg_encoded['labels'])), 
                                                    test_size = 0.10, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state=42)

In [5]:
len(X_train), len(X_val), len(X_test)

(25401, 1411, 1412)

In [9]:
#Dataset
train = Dataset(X_train, y_train)
val = Dataset(X_val, y_val)
test = Dataset(X_test, y_test)

#DataLoader
train_loader = DataLoader(train, batch_size = 64, shuffle=True)
val_loader = DataLoader(val, batch_size = 256, shuffle=True)
test_loader = DataLoader(test, batch_size = 256, shuffle=True)

## Modeling

In [24]:
with open('vocab/word2idx.json') as json_file:
    word2idx = json.load(json_file)
hidden_size = 256
in_neuron = len(word2idx)
padding_idx = word2idx['pad']
loss_fn = nn.CrossEntropyLoss()

cuda = torch.cuda.is_available()
lr = 0.001

In [25]:
model_class = Network(in_neuron, padding_idx, hidden_size = hidden_size, 
                      out_neuron = tax_dict['Class'][-1], num_layers = 2,
                     bidirectional = True)

In [26]:
model_class

Network(
  (embedding): Embedding(74320, 128)
  (lstm): LSTM(128, 256, num_layers=2, bidirectional=True)
  (dropout): Dropout(p=0.13, inplace=False)
  (dense): Linear(in_features=512, out_features=3, bias=True)
)

In [27]:
if cuda:
    model_class.to('cuda')
optimizer = torch.optim.Adam(model_class.parameters(),lr=lr)

EPOCH = 20
for epoch in range(EPOCH):
    train_loss, train_acc = train_network(model_class, train_loader, optimizer, loss_fn, epoch+1, tax = 'Class')
    val_loss,val_acc = evaluate_network(model_class, test_loader, optimizer,loss_fn, tax = 'Class')
    tqdm.write(f'''End of Epoch: {epoch+1}  |  Train Loss: {train_loss:.3f}  |  Val Loss: {val_loss:.3f}  |  Train Acc: {train_acc*100:.2f}%  |  Val Acc: {val_acc*100:.2f}%''')

Epoch: 1:   1%|▎                                                                       | 2/397 [00:00<00:46,  8.43it/s]

Batch : 0/397   Loss : 1.095173954963684   Accuracy : 0.5


Epoch: 1:  13%|█████████▎                                                             | 52/397 [00:05<00:37,  9.20it/s]

Batch : 50/397   Loss : 0.884985625743866   Accuracy : 0.5


Epoch: 1:  26%|█████████████████▉                                                    | 102/397 [00:11<00:33,  8.91it/s]

...

End of Epoch: 2  |  Train Loss: 0.046  |  Val Loss: 0.041  |  Train Acc: 98.83%  |  Val Acc: 98.31%
Batch : 0/397   Loss : 0.014889195561408997   Accuracy : 1.0


Epoch: 3:  13%|█████████▎                                                             | 52/397 [00:05<00:38,  8.95it/s]

Batch : 50/397   Loss : 0.013180038891732693   Accuracy : 1.0


Epoch: 3:  15%|██████████▌                                                            | 59/397 [00:06<00:38,  8.80it/s]


KeyboardInterrupt: 

In [ ]:

out_class = 4
model_class = Network(in_neuron, padding_idx, hidden_size = hidden_size, out_neuron = out_class)

In [ ]:
model_ = Network(in_neuron, padding_idx, hidden_size = hidden_size, out_neuron = out_class)

In [ ]:
model_.load_state_dict(model_class.state_dict())

In [ ]:
out_order = 16
model_.dense = torch.nn.Linear(hidden_size * 2,out_order)

In [ ]:
model_